In [ ]:
# Step 1: Import libraries
import pandas as pd
import numpy as np
import re
from google.colab import files

print(" Starting data preprocessing...")

# Step 2: Upload the file
try:
    uploaded = files.upload()  # You'll be prompted to upload final.csv
except:
    print("File already uploaded.")

# Step 3: Load the data
df_raw = pd.read_csv('final.csv', dtype=str, on_bad_lines='skip')
print(f" Loaded raw data with shape: {df_raw.shape}")

# Step 4: Clean column names
df_raw.columns = [
    re.sub(r'\?.*', '', col.strip().replace('\r\n', ' ').replace('\n', ' ')).strip().lower()
    for col in df_raw.columns
]

# Remove duplicate columns (if any)
df_raw = df_raw.loc[:, ~df_raw.columns.duplicated()]

# Standardize column names
col_map = {
    'timestamp': 'timestamp',
    'by submitting this form, i voluntarily agree to share my academic and behavioral information for research purposes. i understand that my data will be kept confidential and used only for academic analysis related to student scores prediction.':
        'consent',
    'year of study (ex:1st year)': 'year_of_study',
    'number of subject you would like to enter': 'num_subjects'
}
df_raw = df_raw.rename(columns=col_map)

# Step 5: Define block prefixes (for repeated subject entries)
# These are the repeating patterns in your data
blocks = [
    '',           # First subject (no prefix)
    '2.1 ',       # Second subject block
    '3.1 ',       # Third subject block
    '3.2 ',       # Fourth subject block
    '2 ', '3 ', '4 '  # Fallback prefixes
]

# List of all possible subject-related column stems
subject_stems = [
    'subject name',
    'subject credit',
    'what scored did you get for the mid exam  0 - 100',
    ' what is your average score in quizzes or class tests  ',
    ' what marks did you receive for your recent project or coursework  0-100',
    'have you missed any deadlines of the assignment',
    'engagement during lectures',
    'how many hours do you study per day ',
    'do you usually stick to the study schedule',
    'do you follow any additional academic resourcesyou tube ,pdf s ,tutorial, etc '
]

# Build full column list with prefixes
all_cols = []
for prefix in blocks:
    for stem in subject_stems:
        for col in df_raw.columns:
            if stem in col.lower() and prefix in col:
                all_cols.append(col)
                break

# Deduplicate while preserving order
all_cols = list(dict.fromkeys(all_cols))

# Step 6: Helper function to extract numeric values
def clean_score(value):
    if pd.isna(value) or str(value).strip() == '' or str(value).lower() in ['nan', 'no']:
        return np.nan
    value = str(value).strip().lower()

    # Handle ranges
    if 'below 50' in value: return 45
    elif '90-100' in value or '90 to 100' in value: return 95
    elif '80-89' in value: return 84.5
    elif '70-79' in value: return 74.5
    elif '60-69' in value: return 64.5
    elif '50-59' in value: return 54.5
    elif '40-49' in value: return 44.5
    elif '30-39' in value: return 34.5
    elif '20-29' in value: return 24.5
    elif '10-19' in value: return 14.5
    elif '0-9' in value: return 4.5
    else:
        nums = re.findall(r'\d+', value)
        return np.mean([float(x) for x in nums]) if nums else np.nan

# Step 7: Extract all subject entries
subjects_data = []

for idx, row in df_raw.iterrows():
    # Clean year
    year_raw = str(row.get('year_of_study', '')).strip().lower()
    if '1' in year_raw: year = 1
    elif '2' in year_raw: year = 2
    elif '3' in year_raw: year = 3
    elif '4' in year_raw: year = 4
    else: year = 1

    for prefix in blocks:
        # Get subject name
        name_col = None
        for col in df_raw.columns:
            if 'subject name' in col.lower() and prefix in col:
                name_col = col
                break
        if not name_col or name_col not in row or pd.isna(row[name_col]) or str(row[name_col]).strip() in ['', 'nan', 'Subject name']:
            continue

        subject = str(row[name_col]).strip()

        # Helper: safely get field value
        def get_val(stem):
            for col in df_raw.columns:
                if stem in col.lower() and prefix in col:
                    val = row[col]
                    if pd.notna(val) and str(val).strip().lower() not in ['', 'nan']:
                        return val
            return np.nan

        credit = get_val('subject credit')
        mid_exam = clean_score(get_val('mid exam'))
        quiz_avg = clean_score(get_val('quizzes or class tests'))
        project = clean_score(get_val('project or coursework'))
        missed = get_val('missed any deadlines')
        engagement = get_val('engagement during lectures')
        study_hours = get_val('study per day')
        stick = get_val('stick to the study schedule')
        resources = get_val('additional academic resources')

        # Normalize engagement
        engagement_map = {'always': 3, 'usually': 2, 'sometimes': 1, 'rarely': 0, 'not at all': 0}
        engagement = engagement_map.get(str(engagement).strip().lower(), 1)

        # Study hours
        hours_map = {
            'less than 1 hour': 0.5,
            '1-2 hours': 1.5,
            '2-4 hours': 3.0,
            'more than 4 hours': 5.0
        }
        study_hours = hours_map.get(str(study_hours).strip(), 1.0)

        # Binary: Yes/No
        def to_binary(x):
            x = str(x).strip().lower()
            return 1 if x in ['yes', 'always', 'never'] else 0

        missed = to_binary(missed)
        stick = to_binary(stick)
        resources = to_binary(resources)

        # Append subject-level entry
        subjects_data.append({
            'Subject': subject,
            'Year': year,
            'Credit': credit,
            'Mid_Exam_Score': mid_exam,
            'Quiz_Avg_Score': quiz_avg,
            'Project_Score': project,
            'Missed_Deadlines': missed,
            'Engagement': engagement,
            'Study_Hours_Per_Day': study_hours,
            'Stick_To_Schedule': stick,
            'Use_Additional_Resources': resources
        })

# Step 8: Create final DataFrame
df = pd.DataFrame(subjects_data)
print(f"✅ Created {len(df)} subject entries.")

# Convert numeric columns
numeric_cols = ['Credit', 'Mid_Exam_Score', 'Quiz_Avg_Score', 'Project_Score']
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Step 9: Feature Engineering
# Create estimated final score (example weights)
df['Estimated_Final_Score'] = (
    0.4 * df['Mid_Exam_Score'] +
    0.3 * df['Quiz_Avg_Score'] +
    0.3 * df['Project_Score']
)

# Drop rows with no valid scores
df.dropna(subset=['Mid_Exam_Score', 'Quiz_Avg_Score', 'Project_Score'], how='all', inplace=True)
df.reset_index(drop=True, inplace=True)

print(f"✅ Final cleaned data shape: {df.shape}")
print("\nSample of cleaned data:")
print(df.head())

# Step 10: Save and download
output_file = 'cleaned_student_data_for_modeling.csv'
df.to_csv(output_file, index=False)
print(f"\n Saved cleaned data as '{output_file}'")

# Download to your computer
files.download(output_file)
print(" File is downloading to your PC...")

 Starting data preprocessing...


File already uploaded.
 Loaded raw data with shape: (217, 69)
✅ Created 235 subject entries.
✅ Final cleaned data shape: (186, 12)

Sample of cleaned data:
         Subject  Year  Credit  Mid_Exam_Score  Quiz_Avg_Score  Project_Score  \
0            OOP     1     3.0            85.0            84.5           65.0   
1  C programming     1     4.0            50.0            74.5           70.0   
2              C     2     4.0            45.0            54.5           67.0   
3     Electronic     1     3.0            32.0            84.5           65.0   
4           dbms     1     4.0            25.0            84.5           65.0   

   Missed_Deadlines  Engagement  Study_Hours_Per_Day  Stick_To_Schedule  \
0                 1           2                  1.5                  1   
1                 0           2                  1.0                  0   
2                 0           1                  3.0                  1   
3                 1           3                  1.5     

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

 File is downloading to your PC...


In [ ]:
from google.colab import files
import pandas as pd
import numpy as np

# Step 1: Upload the CSV file
print(" Please upload your CSV file...")
uploaded = files.upload()

# Get the filename of the uploaded file
filename = list(uploaded.keys())[0]
print(f" Successfully uploaded: {filename}")

# Step 2: Load the data
df = pd.read_csv(filename)
print(f"\n Dataset Shape: {df.shape}")
print(df.head())

# Step 3: Check for missing values
print("\n Missing values in each column:")
missing = df.isnull().sum()
print(missing)

total_missing = missing.sum()
print(f"\n❗ Total missing values: {total_missing}")

if total_missing > 0:
    print("\n Rows with missing values (first 5):")
    print(df[df.isnull().any(axis=1)].head())
else:
    print("\n No missing values found.")

# Step 4: Handle missing values

# Option 1: Fill numeric columns with mean
numeric_cols = df.select_dtypes(include=[np.number]).columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].mean(numeric_only=True))

# Option 2: Fill categorical columns with mode (most frequent value)
categorical_cols = df.select_dtypes(include=['object']).columns
for col in categorical_cols:
    if df[col].isnull().sum() > 0:
        mode_value = df[col].mode()
        if len(mode_value) > 0:
            df[col] = df[col].fillna(mode_value[0])
        else:
            df[col] = df[col].fillna("Unknown")  # Fallback

print("\n Missing values have been filled.")

# Step 5: Save cleaned data
cleaned_filename = 'cleaned_data_no_missing.csv'
df.to_csv(cleaned_filename, index=False)
print(f"\n Cleaned data saved as: {cleaned_filename}")

# Step 6: Download to your computer
print("\n⬇Downloading cleaned file to your PC...")
files.download(cleaned_filename)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, precision_score, recall_score, f1_score

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [ ]:
# Load the CSV you cleaned earlier
df = pd.read_csv("cleaned_data_no_missing.csv")

print(" Dataset loaded successfully.")
print(" Shape:", df.shape)
df.head()

In [ ]:
# X = all input features
# y = final score (numeric target to predict)

X = df.drop(columns=["Estimated_Final_Score"])  # Make sure column name is correct
y = df["Estimated_Final_Score"]

In [ ]:
categorical_cols = X.select_dtypes(include=["object"]).columns.tolist()
numerical_cols = X.select_dtypes(include=["number"]).columns.tolist()

preprocessor = ColumnTransformer(transformers=[
    ("num", StandardScaler(), numerical_cols),
    ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_cols)
])

# Split data 80% train / 20% test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Apply preprocessing
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

In [ ]:
# MLP (Multilayer Perceptron) with dropout to avoid overfitting

model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_processed.shape[1],)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(1)  # Output layer for regression
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])

model.summary()

In [ ]:
# Trains on 72% of total data, validates on 8% (from training set)
history = model.fit(
    X_train_processed, y_train,
    validation_split=0.1,  # 10% of training for validation
    epochs=100,
    batch_size=32,
    verbose=1
)

In [ ]:
# Visualize how the model learned over time
plt.figure(figsize=(12, 5))

# Plot MSE loss
plt.subplot(1, 2, 1)
plt.plot(history.history["loss"], label="Train Loss")
plt.plot(history.history["val_loss"], label="Val Loss")
plt.title("Loss Curve (MSE)")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()

# Plot MAE
plt.subplot(1, 2, 2)
plt.plot(history.history["mae"], label="Train MAE")
plt.plot(history.history["val_mae"], label="Val MAE")
plt.title("MAE Curve")
plt.xlabel("Epoch")
plt.ylabel("Mean Absolute Error")
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Predict continuous scores
y_pred = model.predict(X_test_processed)

# Regression metrics
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f" Test MSE: {mse:.2f}")
print(f" Test R² Score: {r2:.2f}")

In [ ]:
# Define pass as score >= 50
y_test_binary = (y_test >= 50).astype(int)
y_pred_binary = (y_pred.flatten() >= 50).astype(int)

# Classification metrics
accuracy = accuracy_score(y_test_binary, y_pred_binary)
precision = precision_score(y_test_binary, y_pred_binary)
recall = recall_score(y_test_binary, y_pred_binary)
f1 = f1_score(y_test_binary, y_pred_binary)

print("\n Classification Metrics (Pass/Fail):")
print(f" Accuracy: {accuracy:.2f}")
print(f" Precision: {precision:.2f}")
print(f" Recall: {recall:.2f}")
print(f" F1 Score: {f1:.2f}")
















































































































































































































































































































































































































































































































































































































































































































